## Project summarize reports

### Objective:
- In this project we will combine several Brazilian monetary policy reports provided by the Central Bank and generate a summary of the 3 publications using NLTK resources with the aim of creating a broad summary document of 3 quarters that can be read and researched quickly on all the subject matter covered in the last 9 months on the Brazilian economic scenario.

- Nesse projeto vamos unir vários relatórios de política monetária do Brasil fornecido pelo Banco Central e gerar um resumo das 3 publicações utilizando recursos de NLTK com o objetivo de criar um documento resumido amplo de 3 trimestres que possa ser lido e pesquisado de forma rápida sobre todo o assunto tratado nos últimos 9 meses sobre o cenário econômico brasileiro.

### Data Origin: https://www.bcb.gov.br/publicacoes/rpm/cronologicos
- relatórios de política monetária em PDF: 
politica_monetaria_set2024.pdf
politica_monetaria_dez2024.pdf
politica_monetaria_mar2025.pdf

- The monetary policy report or inflation report is generated by the Central Bank of Brazil every quarter and contains content of great interest to the market:
    - It presents a detailed analysis of the Brazilian and international economic scenario.
    - It discloses the BCB's projections for inflation in the short, medium and long term, considering different scenarios.
    - It explains the reasons behind the decisions taken by the Monetary Policy Committee (COPOM), especially in relation to the basic interest rate (Selic).
    - It provides transparency to the actions and analyses of the Central Bank for the general public, markets and specialists.

- O relatório de política monetária ou relatório de inflação é gerado pelo Banco Central do Brasil a cada trimestre e possui um conteúdo de grande interesse do mercado:
    - Apresenta uma análise detalhada do cenário econômico brasileiro e internacional. 
    - Divulga as projeções do BCB para a inflação no curto, médio e longo prazo, considerando diferentes cenários. 
    - Explica as razões por trás das decisões tomadas pelo Comitê de Política Monetária (COPOM), especialmente em 
    - relação à taxa básica de juros (Selic). 
    - Dá transparência às ações e análises do Banco Central para o público em geral, mercados e especialistas.

- ## Análise exploratória dos dados
- ## Preparação dos dados
- ## Armazenamento dos dados tratados
- ## Geração de sumário

In [96]:
# maximiza nro de linhas e colunas para exibição
# inibe mensagens de warning
import pandas as pd
pd.set_option('display.max_rows', None) # permite a máxima visualização das linhas em um display
pd.set_option('display.max_columns', None) # permite a máxima visualização das colunas em um display
import warnings
warnings.simplefilter('ignore') # inibe a exibição de avisos de warning

In [97]:
import fitz

import re
import nltk

from docx import Document

In [98]:
# converte pdf para txt
def pdf_to_text(pdf_path, txt_path):
    pdf_document = fitz.open(pdf_path)

    with open(txt_path, "w", encoding="utf-8") as text_file:
        for page_number in range(len(pdf_document)):
            page = pdf_document.load_page(page_number)

            text = page.get_text()
            text_file.write(text)

    pdf_document.close()

    return text_file

pdf_path = "dataset/politica_monetaria_set2024.pdf"
txt_path = "treated/politica_monetaria_set2024.txt"
text_file = pdf_to_text(pdf_path, txt_path)

pdf_path2 = "dataset/politica_monetaria_dez2024.pdf"
txt_path2 = "treated/politica_monetaria_dez2024.txt"
text_file2 = pdf_to_text(pdf_path2, txt_path2)

pdf_path3 = "dataset/politica_monetaria_mar2025.pdf"
txt_path3 = "treated/politica_monetaria_mar2025.txt"
text_file3 = pdf_to_text(pdf_path3, txt_path3)

In [99]:
# cria a tabela de frequência de palavras
def _create_frequency_table(text_string) -> dict:

    stopWords = set(stopwords.words("portuguese"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [100]:
# faz a pontuação das frases
def _score_sentences(sentences, freqTable) -> dict:
    sentenceValue = dict()

    for sentence in sentences:
        word_count_in_sentence = (len(word_tokenize(sentence)))
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]

        sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] // word_count_in_sentence

    return sentenceValue

In [101]:
# calcula a pontuação média
def _find_average_score(sentenceValue) -> int:
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original text
    average = int(sumValues / len(sentenceValue))

    return average

In [102]:
# gera o resumo dos textos apurados
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] > (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [103]:
String_com_dados_do_arquivo_aux = ''

def le_arquivo(txt_path):
    global String_com_dados_do_arquivo_aux

    with open(txt_path, encoding="utf-8") as arquivo_lido:        
        String_com_dados_do_arquivo_aux = arquivo_lido.read() 

    return arquivo_lido

In [104]:
arquivo_lido = le_arquivo(txt_path)
String_com_dados_do_arquivo = String_com_dados_do_arquivo_aux

arquivo_lido2 = le_arquivo(txt_path2)
String_com_dados_do_arquivo2 = String_com_dados_do_arquivo_aux

arquivo_lido3 = le_arquivo(txt_path3)
String_com_dados_do_arquivo3 = String_com_dados_do_arquivo_aux

In [105]:
print(arquivo_lido)
print(arquivo_lido2)
print(arquivo_lido3)

<_io.TextIOWrapper name='treated/politica_monetaria_set2024.txt' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='treated/politica_monetaria_dez2024.txt' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='treated/politica_monetaria_mar2025.txt' mode='r' encoding='utf-8'>


In [106]:
def limpa_texto(String_com_dados_do_arquivo):
    article_text = ""

    for line in String_com_dados_do_arquivo:
        article_text += line

    # Removing Square Brackets and Extra Spaces
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
    article_text = re.sub(r'\s+', ' ', article_text)

    # Removing special characters and digits
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

    sentence_list = nltk.sent_tokenize(article_text)
    stopwords = nltk.corpus.stopwords.words('portuguese')

    return article_text

In [107]:
article_text = limpa_texto(String_com_dados_do_arquivo)
article_text2 = limpa_texto(String_com_dados_do_arquivo2)
article_text3 = limpa_texto(String_com_dados_do_arquivo3)

In [108]:
print(article_text)
print(article_text2)
print(article_text3)

Relatório de Inflação Volume 26 | Número 3 | Setembro 2024 ISSN 1517-6576 Relatório de Inflação Volume 26 | Número 3 | Setembro 2024 ISSN 1517-6576 CNPJ 00.038.166/0001-05 Relatório de Inflação | Brasília | v. 26 | nº 3 | set. | 2024 | p. 1‑69 É permitida a reprodução das matérias, desde que mencionada a fonte: Relatório de Inflação, volume 26, nº 3. Convenções estatísticas ... dados desconhecidos. - dados nulos ou indicação de que a rubrica assinalada é inexistente. 0 ou 0,0 menor que a metade do último algarismo, à direita, assinalado. * dados preliminares. O hífen (-) entre anos (1970-1975) indica o total de anos, incluindo-se o primeiro e o último. A barra (/) entre anos (1970/1975) indica a média anual dos anos assinalados, incluindo-se o primeiro e o último, ou, se especificado no texto, o ano-safra ou o ano-convênio. Eventuais divergências entre dados e totais ou variações percentuais são provenientes de arredondamento. Não é citada a fonte dos quadros e dos gráficos de autoria 

In [109]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

def sumariza_relatorio(article_text):
    # 1 Create the word frequency table
    freq_table = _create_frequency_table(article_text)

    '''
    We already have a sentence tokenizer, so we just need 
    to run the sent_tokenize() method to create the array of sentences.
    '''

    # 2 Tokenize the sentences
    sentences = sent_tokenize(article_text)

    # 3 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(sentences, freq_table)

    # 4 Find the threshold
    threshold = _find_average_score(sentence_scores)

    # 5 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.5 * threshold)

    return summary

In [110]:
summary = sumariza_relatorio(article_text)
summary2 = sumariza_relatorio(article_text2)
summary3 = sumariza_relatorio(article_text3)

In [111]:
summary = summary + ' ' + summary2 + ' ' + summary3

In [112]:
print(summary)

 * dados preliminares. ii. • No período de 1999 a 2024, a meta se refere à inflação do ano-calendário. iii. ii. iii. A projeção para o primeiro trimestre de 2026 aumentou 0,2 p.p. em relação ao Relatório anterior. As duas maiores economias recuperam-se do período da pandemia, mas com desequilíbrios. 2/ Argentina, Brasil, Chile, Colômbia, México e Peru. A taxa de desemprego tem se elevado desde o início de 2023, alcançando 4,2% em agosto de 2024, nível ainda historicamente baixo e inferior à estimativa oficial para taxa não cíclica1 (4,4%). A taxa de participação tem se mantido estável, após recuperação nos últimos anos, em níveis inferiores aos de 2019. A maior contribuição veio do setor externo (+0,5 p.p.). A formação bruta de capital fixo contraiu 2,2% (contribuição de -0,5 p.p.). Entre as cinco maiores economias, Holanda e Espanha registraram crescimento mais elevado (1,0% T/T e 0,8% T/T, respectivamente). França e Itália cresceram à mesma taxa do bloco, de 0,2% T/T. A Alemanha regi

In [113]:
# grava o sumário em arquivo word
# Create a new Document
doc = Document()

# Add a paragraph with the string
doc.add_paragraph(summary)

# Save the document
doc.save("treated/politica_monetaria_set2024_dez2004_mar2025.docx")